In [1]:
from mpl_toolkits.mplot3d import Axes3D

import os
import h5py
import numpy as np
import matplotlib.pyplot as plt

import SRepClass as srep

In [2]:
np.random.seed( seed=2718)

In [3]:
train_ratio = 0.8

neg_folder = '../refined_sreps/salt/neg/v06Correct/'
pos_folder = '../refined_sreps/salt/pos/v06Correct/'

h5_filename = './data/medicalnet02_cloud.h5'

In [4]:
def get_srep_pnts( filename ):
    srep_obj = srep.SRep( filename )
    
    upPnts = srep_obj.upPoints[1::2,:]
    dnPnts = srep_obj.downPoints[1::2,:]
    crPnts = srep_obj.crestPoints[1::2,:]
    
    allPnts = np.concatenate( [upPnts,dnPnts,crPnts], axis=0 )
    
    return np.random.permutation( allPnts ) # we don't want the network to learn something order related

In [5]:
def pnts_from_dir( folder ):
    # TODO:
    # check for folder existance
    sfolders = os.listdir( folder )
    # TODO:
    # should check sfolders for empty
    
    allPnts = []
    for sfold in sfolders:
        # Assumption-01:
        # pnts is always the same size
        pnts = get_srep_pnts( os.path.join( folder, sfold, "header.xml" ) )
        
        allPnts.append( pnts )
    
    # This will break if Assumption-01 is not met
    return np.array( allPnts )

In [6]:
neg_pnts = pnts_from_dir( neg_folder )
pos_pnts = pnts_from_dir( pos_folder )

In [7]:
neg_train_num = int( 0.8 * np.shape( neg_pnts )[0] )
pos_train_num = int( 0.8 * np.shape( pos_pnts )[0] )

neg_pnts_trn = neg_pnts[0:neg_train_num]
neg_pnts_tst = neg_pnts[neg_train_num:]
neg_lbls_trn = np.zeros( (neg_train_num), int )
neg_lbls_tst = np.zeros( (len(neg_pnts_tst)), int )

pos_pnts_trn = pos_pnts[0:pos_train_num]
pos_pnts_tst = pos_pnts[pos_train_num:]
pos_lbls_trn = np.ones( (pos_train_num), int )
pos_lbls_tst = np.ones( (len(pos_pnts_tst)), int )

In [8]:
trn_data = np.concatenate( (neg_pnts_trn, pos_pnts_trn), axis=0 )
trn_lbls = np.concatenate( (neg_lbls_trn, pos_lbls_trn), axis=0 )

trn_ind = np.random.permutation( len( trn_data ) )
trn_data = trn_data[trn_ind]
trn_lbls = trn_lbls[trn_ind]

In [9]:
tst_data = np.concatenate( (neg_pnts_tst, pos_pnts_tst), axis=0 )
tst_lbls = np.concatenate( (neg_lbls_tst, pos_lbls_tst), axis=0 )

tst_ind = np.random.permutation( len( tst_data ) )
tst_data = tst_data[tst_ind]
tst_lbls = tst_lbls[tst_ind]

In [10]:
err

NameError: name 'err' is not defined

In [ ]:
with h5py.File( h5_filename, 'w' ) as hf:
    hf.create_dataset( 'tr_cloud',  data=trn_data )
    hf.create_dataset( 'tr_labels', data=trn_lbls )
    
    hf.create_dataset( 'test_cloud',  data=tst_data )
    hf.create_dataset( 'test_labels', data=tst_lbls )